Using a calibration pattern (e.g., a checkerboard), perform camera 
calibration to obtain intrinsic and extrinsic parameters. Display the 
calibration results and the reprojection error.

In [4]:
import numpy as np
import cv2
import glob

# Configuration - adjust these values
images_folder = "checkboa"  # Folder containing checkerboard images
pattern_size = (6, 9)                 # Inner corners (rows, columns)
square_size = 0.025                   # Square size in meters (25mm)

# Prepare 3D object points (real world coordinates)
objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size  # Scale to real-world size

# Arrays to store points
object_points = []  # 3D points in real world
image_points = []   # 2D points in image plane

# Find all calibration images
images = glob.glob(f'{images_folder}/*.jpg') + glob.glob(f'{images_folder}/*.png')

# Process each image
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find chessboard corners
    found, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    
    if found:
        object_points.append(objp)
        
        # Refine corner positions
        corners_refined = cv2.cornerSubPix(
            gray, corners, (11, 11), (-1, -1),
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        image_points.append(corners_refined)
        
        # Visualize detected corners
        cv2.drawChessboardCorners(img, pattern_size, corners_refined, found)
        cv2.imshow('Detected Corners', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Perform camera calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    object_points, image_points, gray.shape[::-1], None, None
)

# Display results
print("=== Calibration Results ===")
print(f"Re-projection Error: {ret:.4f} (lower is better)")

print("\nCamera Matrix (Intrinsic Parameters):")
print("[[fx  0  cx]")
print(" [ 0 fy  cy]")
print(" [ 0  0   1]]")
print(camera_matrix)

print("\nDistortion Coefficients (k1, k2, p1, p2, k3):")
print(dist_coeffs)

print("\nFirst Extrinsic Parameters:")
print("Rotation Vector:", rvecs[0].flatten())
print("Translation Vector:", tvecs[0].flatten())

# Save calibration data for later use
np.savez("camera_calibration.npz",
         camera_matrix=camera_matrix,
         dist_coeffs=dist_coeffs)

=== Calibration Results ===
Re-projection Error: 4.9950 (lower is better)

Camera Matrix (Intrinsic Parameters):
[[fx  0  cx]
 [ 0 fy  cy]
 [ 0  0   1]]
[[ 631.38880826    0.         -211.80045351]
 [   0.          442.74484021 -116.37055388]
 [   0.            0.            1.        ]]

Distortion Coefficients (k1, k2, p1, p2, k3):
[[ 1.43434672e-03  1.09446006e-03 -2.32401954e-02 -2.08449892e-02
  -3.80947269e-05]]

First Extrinsic Parameters:
Rotation Vector: [ 0.1495812  -0.05327068 -1.56149978]
Translation Vector: [0.06913892 0.2019305  0.07179684]


In [5]:
import numpy as np
import cv2
import glob

# Configuration
images_folder = "checkboa"  # Folder containing checkerboard images
pattern_size = (6, 9)  # Inner corners (rows, columns)
square_size = 0.025    # Square size in meters

# Prepare 3D points
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:,:2] = np.mgrid[:pattern_size[0],:pattern_size[1]].T.reshape(-1,2) * square_size

# Find corners in all images
obj_points, img_points = [], []
for fname in glob.glob(f'{images_folder}/*.jpg') + glob.glob(f'{images_folder}/*.png'):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    if ret:
        corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), 
                                 (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        obj_points.append(objp)
        img_points.append(corners)

# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

# Calculate reprojection error
mean_error = sum(cv2.norm(img_points[i], 
                cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)[0], 
                cv2.NORM_L2)/len(img_points[i]) 
               for i in range(len(obj_points))) / len(obj_points)

# Results
print(f"Calibration Error: {ret:.3f} (RMS), Mean Reprojection: {mean_error:.3f} pixels")
print("Camera Matrix:\n", mtx)
print("Distortion:", dist.flatten())
np.savez("calibration.npz", mtx=mtx, dist=dist)

Calibration Error: 4.995 (RMS), Mean Reprojection: 0.649 pixels
Camera Matrix:
 [[ 631.38880826    0.         -211.80045351]
 [   0.          442.74484021 -116.37055388]
 [   0.            0.            1.        ]]
Distortion: [ 1.43434672e-03  1.09446006e-03 -2.32401954e-02 -2.08449892e-02
 -3.80947269e-05]
